In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from collections import Counter

import nltk
nltk.download('all')

/Users/kevinyang/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/kevinyang/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/kevinyang/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/kevinyang/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/kevinyang/nl

[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /Users/kevinyang/nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /Users/kevinyang/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /Users/kevinyang/nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /Users/kevinyang/nltk_data...
[nltk_data]    |   Package sinica_treebank is already up-to-date!
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /Users/kevinyang/nltk_data...
[nltk_data]    |   Package smultron is already up-to-date!
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /Users/kevinyang/nltk_data.

True

Importing and trimming twitter sentiment dataset

Insert header, remove numbers, usernames, and NO_QUERY

Move first row that became header down to data, add headers

In [2]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding = "ISO-8859-1",names=["score","id","datetime","NO_QUERY","usernames","tweet"])

df = df.drop(['id','NO_QUERY','usernames'],axis=1)

df.head()

,score,datetime,tweet
0,0,Mon Apr 06 22:19:45 PDT 2009,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,Mon Apr 06 22:19:49 PDT 2009,is upset that he can't update his Facebook by ...
2,0,Mon Apr 06 22:19:53 PDT 2009,@Kenichan I dived many times for the ball. Man...
3,0,Mon Apr 06 22:19:57 PDT 2009,my whole body feels itchy and like its on fire
4,0,Mon Apr 06 22:19:57 PDT 2009,"@nationwideclass no, it's not behaving at all...."


Checking count values, dataset listed from 0 = negative to 4 = positive. 

In [3]:
df.shape

(1600000, 3)

In [4]:
df['score'].value_counts()

4    800000
0    800000
Name: score, dtype: int64

Clean up links, @users, hastags

In [5]:
stopwords = nltk.corpus.stopwords.words("english")

def cleanup(text):
    clean = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(text)).split()
    tokens = []
    for token in clean:
        if token not in stopwords:
            tokens.append(token)
    return tokens

In [6]:
df['tweet'] = df['tweet'].apply(lambda x: cleanup(x))

df['score'] = df['score'].replace([0,4],['neg','pos'])

df.head()

,score,datetime,tweet
0,neg,Mon Apr 06 22:19:45 PDT 2009,"[Awww, bummer, You, shoulda, got, David, Carr,..."
1,neg,Mon Apr 06 22:19:49 PDT 2009,"[upset, update, Facebook, texting, might, cry,..."
2,neg,Mon Apr 06 22:19:53 PDT 2009,"[I, dived, many, times, ball, Managed, save, 5..."
3,neg,Mon Apr 06 22:19:57 PDT 2009,"[whole, body, feels, itchy, like, fire]"
4,neg,Mon Apr 06 22:19:57 PDT 2009,"[behaving, mad, I, see]"


In [7]:
def to_tuple(x):
    subset = x[['tweet','score']]
    tuples = [tuple(i) for i in subset.to_numpy()]
    return tuples

In [8]:
documents = to_tuple(df)
documents[0]

(['Awww',
  'bummer',
  'You',
  'shoulda',
  'got',
  'David',
  'Carr',
  'Third',
  'Day',
  'D'],
 'neg')

In [9]:
def word_master(x):
    master_list = []
    for i in range(len(x)):
        master_list += x['tweet'][i]
    return master_list

words = word_master(df)

#word_features = nltk.FreqDist(w.lower() for w in words).most_common(2000)
#word_features[0:5]

word_features = nltk.FreqDist(w.lower() for w in words).most_common(500)
word_features = [word_tuple[0] for word_tuple in word_features]
word_features

['i',
 'good',
 'day',
 'get',
 'like',
 'go',
 'quot',
 'today',
 'work',
 'love',
 'going',
 'got',
 'lol',
 'time',
 'back',
 'u',
 'one',
 'know',
 'im',
 'really',
 'amp',
 'see',
 'night',
 'still',
 'it',
 '2',
 'well',
 'new',
 'want',
 'think',
 'home',
 'thanks',
 'oh',
 'much',
 'miss',
 'need',
 'last',
 'morning',
 'tomorrow',
 'hope',
 'great',
 'my',
 'the',
 '3',
 'twitter',
 'you',
 'haha',
 'feel',
 'sad',
 'fun',
 'just',
 'wish',
 'sleep',
 'right',
 'bad',
 'would',
 'happy',
 'sorry',
 'tonight',
 'come',
 'make',
 'way',
 'getting',
 'though',
 'gonna',
 'nice',
 'better',
 'watching',
 'yeah',
 'so',
 'wait',
 'bed',
 'could',
 'week',
 'and',
 'school',
 'people',
 'but',
 'hate',
 'days',
 'even',
 'hey',
 '4',
 'yes',
 'next',
 'no',
 'lt',
 'weekend',
 'awesome',
 'dont',
 'thank',
 'that',
 'never',
 'soon',
 'cant',
 'long',
 'take',
 'little',
 'working',
 'first',
 'wanna',
 'please',
 'say',
 'best',
 'tired',
 'everyone',
 'show',
 'ok',
 'life',
 'sic

In [10]:
def document_features(document):    
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains('+ word +')'] = (word in document_words)
    return features

featuresets = [(document_features(d), c) for (d,c) in documents]

In [11]:
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier, test_set))

classifier.show_most_informative_features(10)

0.6
Most Informative Features
           contains(sad) = True              neg : pos    =     18.0 : 1.0
         contains(hurts) = True              neg : pos    =     15.9 : 1.0
      contains(headache) = True              neg : pos    =     12.4 : 1.0
       contains(welcome) = True              pos : neg    =     11.7 : 1.0
          contains(sick) = True              neg : pos    =     10.3 : 1.0
         contains(sucks) = True              neg : pos    =      9.9 : 1.0
           contains(www) = True              pos : neg    =      9.7 : 1.0
          contains(poor) = True              neg : pos    =      8.9 : 1.0
           contains(ugh) = True              neg : pos    =      7.9 : 1.0
         contains(broke) = True              neg : pos    =      7.3 : 1.0
